
# Local


In [1]:
! pip install -r ../requirements.txt

import sys
import os
sys.path.append(os.path.expanduser("~/repos/NeurOps/pytorch"))


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


# Colab


In [ ]:
! git clone https://github.com/SuReLI/NeurOps.git
! git clone https://github.com/k8lion/GrowAndPrune.git
%cd GrowAndPrune/src
! pip install --quiet -r ../requirements.txt

import sys
import os
sys.path.append("../../NeurOps/pytorch")

Cloning into 'NeurOps'...
remote: Enumerating objects: 517, done.
remote: Counting objects: 100% (517/517), done.
remote: Compressing objects: 100% (311/311), done.
remote: Total 517 (delta 276), reused 413 (delta 192), pack-reused 0
Receiving objects: 100% (517/517), 261.02 KiB | 13.74 MiB/s, done.
Resolving deltas: 100% (276/276), done.
Cloning into 'GrowAndPrune'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 90 (delta 36), reused 69 (delta 21), pack-reused 0
Unpacking objects: 100% (90/90), 9.49 MiB | 5.71 MiB/s, done.
/content/GrowAndPrune/src
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 18.9 MB/s eta 0:00:00


# Imports

In [2]:
import numpy as np
import torch
import copy
import math
import matplotlib.pyplot as plt
from celluloid import Camera
from IPython.display import HTML


from neurops import *

from growprune import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/kaitlin/repos/GrowAndPrune/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kaitlin/repos/GrowAndPrune/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [14]:
data = TransferToyData()
train_loader, val_loader, test_loader = split_dataset(data, val_size=0.1, test_size=0.1, batch_size=32)

In [15]:
model = ModMLP(2,2,1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = torch.nn.BCEWithLogitsLoss()

In [19]:
for _ in range(10):
    _,_,_ = train(model, train_loader, optimizer, criterion, epochs=100, val_loader=val_loader, device=device, verbose=False, val_verbose=False, val_acts=True)
    test(model, val_loader, criterion, device=device, verbose=True)

Average loss: 0.0031, Accuracy: 100/100 (100.00%)


KeyboardInterrupt: 

In [21]:
for angle in range(0, 180, 5):
    print("Angle: ", angle, end=" ")
    newdata = TransferToyData(angle=math.radians(angle))
    train_loader, val_loader, test_loader = split_dataset(newdata, val_size=0.1, test_size=0.1, batch_size=32)
    test(model, val_loader, criterion, device=device, verbose=True)

Angle:  0 Average loss: 0.0025, Accuracy: 100/100 (100.00%)
Angle:  5 Average loss: 0.0042, Accuracy: 100/100 (100.00%)
Angle:  10 Average loss: 0.0018, Accuracy: 100/100 (100.00%)
Angle:  15 Average loss: 0.0031, Accuracy: 99/100 (99.00%)
Angle:  20 Average loss: 0.0031, Accuracy: 99/100 (99.00%)
Angle:  25 Average loss: 0.0037, Accuracy: 100/100 (100.00%)
Angle:  30 Average loss: 0.0033, Accuracy: 100/100 (100.00%)
Angle:  35 Average loss: 0.0027, Accuracy: 100/100 (100.00%)
Angle:  40 Average loss: 0.0033, Accuracy: 100/100 (100.00%)
Angle:  45 Average loss: 0.0028, Accuracy: 100/100 (100.00%)
Angle:  50 Average loss: 0.0049, Accuracy: 98/100 (98.00%)
Angle:  55 Average loss: 0.0051, Accuracy: 99/100 (99.00%)
Angle:  60 Average loss: 0.0066, Accuracy: 100/100 (100.00%)
Angle:  65 Average loss: 0.0051, Accuracy: 100/100 (100.00%)
Angle:  70 Average loss: 0.0066, Accuracy: 100/100 (100.00%)
Angle:  75 Average loss: 0.0073, Accuracy: 100/100 (100.00%)
Angle:  80 Average loss: 0.0115, A